In [1]:
import sys
from pathlib import Path

project_root = Path.cwd().parents[0]  # importing functions from other folders
sys.path.insert(0, str(project_root))

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")  # non-interactive backend
import matplotlib.pyplot as plt
import os
from _data.data_utils import read_in
from _fitting.fitting_utils import hist_plot, CI_plot, CI_plot_alt, CI_plot_both, plot_posteriors_side_by_side, plot_spline
from _fitting.model_utils import build_model, data_settings_to_name, model_settings_to_name, abbrev_stat, settings_to_var_names, elpd_to_xr, elpd_to_row, model_fit, create_html_report
import pymc as pm
import arviz as az
from patsy import dmatrix
import nutpie
import time
from pprint import pprint
from IPython.display import display
import seaborn as sns
from pymc.variational.callbacks import CheckParametersConvergence
import xarray as xr
import pickle
import warnings

az.style.use("arviz-darkgrid")


if '___laptop' in os.listdir('../'):
    # laptop folder
    folder = "../../_data/p-dengue/"
elif '___server' in os.listdir('../'):
    # server folder
    folder = "../../../../../data/lucaratzinger_data/p_dengue/"

In [2]:
data_settings = {'admin':2, 'max_lag':6, 'start_year':2016, 'start_month':1, 'end_year':2019, 'end_month':12}
data_name = data_settings_to_name(data_settings)
data = read_in(folder, **data_settings, standardise=True, dropna=True, celsius=True, tp_log=True)
print('data: ',data_name)

data:  a2_201601_201912


In [3]:
if '___laptop' in os.listdir('../'):
    # laptop folder
    outpath = "../../_data/p-dengue/model_fits/"
elif '___server' in os.listdir('../'):
    # server folder
    outpath = "../../../../../data/lucaratzinger_data/p_dengue/model_fits"

In [4]:
model_dict = {}
for surv in [None, 'surveillance_pop_weighted', 'urban_surveillance_pop_weighted'][0:2]:
    settings = {
            'surveillance_name':surv,
            'urbanisation_name':None,
            'stat_names':['tp_24hmean_pop_weighted_log(0)'], 'degree':3, 'num_knots':5, 'knot_type':'quantile','orthogonal':True}
    model_dict[model_settings_to_name(settings)] = settings

for model_name, model_settings in model_dict.items():
    print('model: ', model_name)
    model_fit(data, data_name, model_settings, outpath, n_chains=2, n_draws=20, n_tune=50, invert_log=True)
    break


Only 20 samples per chain. Reliable r-hat and ESS diagnostics require longer chains for accurate estimate.


model:  [nosurv__nourb][tp_24hmean_log_p(0)][3,5,q,o]

Posterior Sampling 10.06 seconds
Log Likelihood Compute 0.37 seconds 

HTML reports written to: ../../../../../data/lucaratzinger_data/p_dengue/model_fits/a2_201601_201912/outputs/[nosurv__nourb][tp_24hmean_log_p(0)][3,5,q,o]/report_[[nosurv__nourb][tp_24hmean_log_p(0)][3,5,q,o]].html, ../../../../../data/lucaratzinger_data/p_dengue/model_fits/a2_201601_201912/reports/report_[[nosurv__nourb][tp_24hmean_log_p(0)][3,5,q,o]].html
